In [ ]:
%matplotlib inline

In [ ]:
# Author: Qingzhou Li and Leo Zhang based on https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html by Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
!pip install seqeval
from seqeval.metrics import f1_score, classification_report
torch.manual_seed(1)
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=8ecce6ec5c9e33647ae9d9894e745cc8091a147b2505dafeee323144d713cb03
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


Helper functions to make the code more readable.



In [ ]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# (This is for CRF model) Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))



## Bi-LSTM for NER (Toy sample)

Create model



In [ ]:
class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        #self.char_embeds = nn.xxxxxxxx
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))



    def cross_entropy_loss(self, sentence, tags):
        feats = self.forward(sentence)
        loss_fuction = nn.CrossEntropyLoss()
        loss = loss_fuction(feats, tags)
        return loss


    def forward(self, sentence):
        # Get the emission scores from the BiLSTM
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

        # Find the best path, given the features.

        return lstm_feats

    def predict(self, sentence):
        ix_pred = np.argmax(self.forward(sentence).numpy(), axis=1)
        ix_to_tag = {v:k for k, v in self.tag_to_ix.items()}
        tag_pred = [ix_to_tag[i] for i in ix_pred]
        return tag_pred

In [ ]:
# Import the trainning data
training_entity = pd.read_csv('entity.csv')
with open("sents_dict.txt", "r") as fp:
  sents_dict = json.load(fp)

In [ ]:
# Format the trainning data with IOB tagging
training_data1 = []
for key in sents_dict:
  iob = ["O"] * len(sents_dict[key].split())
  current_table = training_entity[training_entity["sent_num"] == key]
  for index, row in current_table.iterrows():
    start = row['start']
    end = row['end']
    concept = row['concept']
    if start == (end - 1):
      iob = iob[:start] + ['B-'+concept] + iob[end:]
    else:
      num_i = end - start - 1
      iob = iob[:start] + ['B-'+concept] + ['I-'+concept] * num_i + iob[end:]
  training_data1.append((key,sents_dict[key].split(),iob))

In [ ]:
# Import the test data
test_entity = pd.read_csv('test_entity.csv')
with open("test_sents_dict.txt", "r") as fp:
  test_sents_dict = json.load(fp)

In [ ]:
# Format the test data with IOB tagging
test_data = []
for key in test_sents_dict:
  iob = ["O"] * len(test_sents_dict[key].split())
  current_table = test_entity[test_entity["sent_num"] == key]
  for index, row in current_table.iterrows():
    start = row['start']
    end = row['end']
    concept = row['concept']
    if start == (end - 1):
      iob = iob[:start] + ['B-'+concept] + iob[end:]
    else:
      num_i = end - start - 1
      iob = iob[:start] + ['B-'+concept] + ['I-'+concept] * num_i + iob[end:]
  test_data.append((key,test_sents_dict[key].split(),iob))

In [ ]:
# Split training data into train and validation 80:20
train, val = train_test_split(training_data1, test_size=0.2, random_state=1234)

In [ ]:
class EarlyStopping:
    def __init__(self, tolerance=5, min_delta=0):
        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.tolerance:
                return True
        return False

Run training



In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 64

training_data = train
validation_data = val
word_to_ix = {}
for key, sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
for key, sentence, tags in validation_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
for key, sentence, tags in test_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"O": 0, START_TAG: 1, STOP_TAG: 2, "B-Drug": 3, "B-Route": 4, "B-Reason": 5, "B-Duration": 6, "B-Dosage": 7, "B-Frequency": 8, "B-Strength": 9,
             "B-Form": 10, "B-ADE": 11, "I-Drug": 12, "I-Route": 13, "I-Reason": 14, "I-Duration": 15, "I-Dosage": 16, "I-Frequency": 17,
             "I-Strength": 18, "I-Form": 19, "I-ADE": 20
             }
model = BiLSTM( len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)
early_stopping = EarlyStopping(tolerance=5, min_delta=0)
train_loss = []
validation_loss = []
epoch_i = 0
# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[1][1], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[1][2]], dtype=torch.float)
    print('label',precheck_tags)

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for i in tqdm(range(150)):
    for key, sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.cross_entropy_loss(sentence_in, targets)


        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

    # split into training and validation, remember the loss in validation find the lowest validation loss, early stop.
    # performance measure on validation data.
    train_loss.append(loss.item())
    with torch.no_grad():
      for key, sentence, tags in validation_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long, requires_grad=False)

        # Step 3. Run our forward pass.
        val_loss = model.cross_entropy_loss(sentence_in, targets)

    epoch_i += 1
    # early stopping
    if early_stopping.early_stop(val_loss):
      print("We are at epoch:", epoch_i)
      break
    validation_loss.append(val_loss.item())


with torch.no_grad():
  precheck_sent = prepare_sequence(training_data[1][1], word_to_ix)
  print('prediction', np.argmax(model(precheck_sent).numpy(), axis = 1))


label tensor([ 0.,  3., 12.,  9., 18.,  4.])


 11%|█         | 16/150 [05:16<44:11, 19.79s/it]

We are at epoch: 17
prediction [ 0  3 12  9 18  4]


In [ ]:
# Individual NER model performance measure
pred = []
true = []
for i in test_data:
  with torch.no_grad():
    prepare_sent = prepare_sequence(i[1], word_to_ix)
    y_pred = model.predict(sentence= prepare_sent)
    pred.append(y_pred)
    true.append(i[2])

print(f1_score(true,pred))
print(classification_report(true, pred))

0.7942392566782811
              precision    recall  f1-score   support

         ADE       0.30      0.07      0.11       625
      Dosage       0.87      0.80      0.84      2681
        Drug       0.82      0.81      0.81     10575
    Duration       0.57      0.50      0.53       378
        Form       0.92      0.87      0.89      4354
   Frequency       0.82      0.74      0.78      4012
      Reason       0.52      0.54      0.53      2545
       Route       0.89      0.89      0.89      3513
    Strength       0.78      0.81      0.79      4230

   micro avg       0.81      0.78      0.79     32913
   macro avg       0.72      0.67      0.69     32913
weighted avg       0.80      0.78      0.79     32913



In [ ]:
from collections import defaultdict

def list_duplicates(seq):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(i)
    return ((key,locs) for key,locs in tally.items() if len(locs)>=1)



In [ ]:
# Identify predicted entities with their respective position in a sentence
out = []
for key, sentence, tags in test_data:
  with torch.no_grad():
    prepare_sent = prepare_sequence(sentence, word_to_ix)
    pred_entity = model.predict(sentence= prepare_sent)
    relation = [key,sentence]
    sample = []
    indices = [i for i, x in enumerate(pred_entity) if x == "B-Drug"]
    for i in indices:
      sec_index = i+1
      if i != (len(pred_entity)-1):
        while sec_index < len(pred_entity) and pred_entity[sec_index] == 'I-Drug':
          sec_index += 1
      drug_index = [i,sec_index]

      for item in sorted(list_duplicates(pred_entity)):
        if item[0][0] == 'B' and item[0] != 'B-Drug':
          for i in item[1]:
            seco_index = i+1
            if i != (len(pred_entity)-1):
              while seco_index < len(pred_entity) and pred_entity[seco_index] == ''.join(['I-',item[0][2:]]):
                seco_index += 1
            entity_index = [i,seco_index]
            sample.append([entity_index, drug_index, item[0][2:]+'-'+'Drug'])
    relation.append(sample)
    out.append(relation)

In [ ]:
# Import the gold standard test data
with open("test_relation.txt", "r") as fp:
  test_relation = json.load(fp)

In [ ]:
# Identify TP, FP, FN in the relations formed from predicted entities
TP = []
FP = []
FN = []
for key1, sentence1, relation1 in out:
  for key2, sentence2, relation2 in test_relation:
    if key1 == key2 and sentence1 == sentence2:
      for i in relation1:
        if i not in relation2:
          FP.append(i)
        elif i in relation2:
          TP.append(i)
      for i in relation2:
        if i not in relation1:
          FN.append(i)

In [ ]:
print(len(TP))
print(len(FP))
print(len(FN))

15158
31435
6920


In [ ]:
relation_out = []
for key1, sentence1, relation1 in out:
  rela = []
  for key2, sentence2, relation2 in test_relation:
    if key1 == key2 and sentence1 == sentence2:
      for i in relation1:
        if i in relation2:
          rela.append(i)
      relation_out.append([key1,sentence1,rela])

In [ ]:
print(relation_out[0:10])

[['0-1', ['He', 'received', '2mg', 'IV', 'ativan', 'with', 'somnolence', 'and', 'apenea'], [[[6, 7], [4, 5], 'ADE-Drug'], [[3, 4], [4, 5], 'Route-Drug'], [[2, 3], [4, 5], 'Strength-Drug']]], ['0-6', ['Patient', 'developed', 'SVT', 'in', 'the', 'cath', 'lab', 'and', 'was', 'treated', 'with', '9', 'mg', 'IV', 'metoprolol'], [[[13, 14], [14, 15], 'Route-Drug'], [[11, 13], [14, 15], 'Strength-Drug']]], ['0-11', ['Evidence', 'of', 'right', 'heart', 'strain,', 'manifest', 'as', 'reflux', 'of', 'IV', 'contrast'], [[[9, 10], [10, 11], 'Route-Drug']]], ['0-15', ['He', 'was', 'started', 'on', 'heparin', 'ggt', 'in', 'house', 'and', 'transitioned', 'to', 'an', 'anticoagulation', 'regimen', 'of', 'coumadin', ',', 'with', 'a', 'lovenox'], [[[5, 6], [4, 5], 'Route-Drug']]], ['0-22', ['He', 'was', 'maintained', 'on', 'a', 'valium', 'CIWA', 'and', 'klonipin', '1mg', 'TID'], [[[10, 11], [8, 9], 'Frequency-Drug'], [[9, 10], [8, 9], 'Strength-Drug']]], ['0-27', ['Psych', 'was', 'consulted', 'who', 'felt'

In [ ]:
import pickle
with open("re_test", "wb") as fp:
   pickle.dump(relation_out, fp)

In [ ]:
# Format end-to-end relations for RE model testing
output_relation = []
for i in relation_out:
  output_relation.append(('re',i))

## Hyperparameter tuning

Baseline: SGD, lr=0.01, hidden dim=4, tolerance=5

baseline: weighted avg f1: 0.59 117 epochs

SGD, lr=0.01, hidden dim=64, tolerance=5: 0.66 32 epochs
                       0.80 28 epochs
                       0.68 42
                       0.70 51
                       0.68 41

SGD, lr=0.01, hidden dim=4, tolerance=5: 0.67 37 epochs
                       0.66 38 epochs
                       0.50 57
                       0.42 37

SGD, lr=0.01, hidden dim=4, tolerance=10: 0.75 83 epochs
                       0.74 78 epochs

SGD, lr=0.1, hidden dim=64, tolerance=5: 0.82 38 epochs

SGD, lr=0.2, hidden dim=64, tolerance=5: 0.82 22 epochs

SGD, lr=0.3, hidden dim=64, tolerance=5: 0.83 16 epochs

SGD, lr=0.3, hidden dim=64, tolerance=8: 0.84 24 epochs

SGD, lr=0.4, hidden dim=64, tolerance=5: 0.82 13 epochs

SGD, lr=0.01, hidden dim=128, tolerance=5: 0.76 12 epochs




## BiLSTM + CRF for NER






For this section, we will see a full, complicated example of a Bi-LSTM
Conditional Random Field for named-entity recognition. The LSTM tagger
above is typically sufficient for part-of-speech tagging, but a sequence
model like the CRF is really essential for strong performance on NER.
Familiarity with CRF's is assumed. Although this name sounds scary, all
the model is a CRF but where an LSTM provides the features. This is
an advanced model though, far more complicated than any earlier model in
this tutorial. If you want to skip it, that is fine. To see if you're
ready, see if you can:

-  Write the recurrence for the viterbi variable at step i for tag k.
-  Modify the above recurrence to compute the forward variables instead.
-  Modify again the above recurrence to compute the forward variables in
   log-space (hint: log-sum-exp)

If you can do those three things, you should be able to understand the
code below. Recall that the CRF computes a conditional probability. Let
$y$ be a tag sequence and $x$ an input sequence of words.
Then we compute

\begin{align}P(y|x) = \frac{\exp{(\text{Score}(x, y)})}{\sum_{y'} \exp{(\text{Score}(x, y')})}\end{align}

Where the score is determined by defining some log potentials
$\log \psi_i(x,y)$ such that

\begin{align}\text{Score}(x,y) = \sum_i \log \psi_i(x,y)\end{align}

To make the partition function tractable, the potentials must look only
at local features.

In the Bi-LSTM CRF, we define two kinds of potentials: emission and
transition. The emission potential for the word at index $i$ comes
from the hidden state of the Bi-LSTM at timestep $i$. The
transition scores are stored in a $|T|x|T|$ matrix
$\textbf{P}$, where $T$ is the tag set. In my
implementation, $\textbf{P}_{j,k}$ is the score of transitioning
to tag $j$ from tag $k$. So:

\begin{align}\text{Score}(x,y) = \sum_i \log \psi_\text{EMIT}(y_i \rightarrow x_i) + \log \psi_\text{TRANS}(y_{i-1} \rightarrow y_i)\end{align}

\begin{align}= \sum_i h_i[y_i] + \textbf{P}_{y_i, y_{i-1}}\end{align}

where in this second expression, we think of the tags as being assigned
unique non-negative indices.

If the above discussion was too brief, you can check out
[this](http://www.cs.columbia.edu/%7Emcollins/crf.pdf)_ write up from
Michael Collins on CRFs.

## Implementation Notes

The example below implements the forward algorithm in log space to
compute the partition function, and the viterbi algorithm to decode.
Backpropagation will compute the gradients automatically for us. We
don't have to do anything by hand.

The implementation is not optimized. If you understand what is going on,
you'll probably quickly see that iterating over the next tag in the
forward algorithm could probably be done in one big operation. I wanted
to code to be more readable. If you want to make the relevant change,
you could probably use this tagger for real tasks.


Create model

In [ ]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

```
# This is formatted as code
```



        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

Running Training

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(
        300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))
# We got it!